# DCA Ingest - DROID Analysis

Dieses Notebook führt eine DROID-Analyse auf einem Ordner durch und speichert die Ergebnisse als CSV.

## Imports und Setup

In [4]:
import os
import subprocess

## Konfiguration

Hier werden alle Pfade und Parameter definiert.

In [5]:
# Dynamisch das Home-Verzeichnis holen
home_dir = os.path.expanduser("~")
base_path = os.path.join(home_dir, "work")

# Dataset-Name definieren
dataset_to_analyze = "2104_mockup"

# Pfade definieren
droid_script_path = os.path.join(base_path, "27_DCA_Ingest/src/droid-binary-6.7.0-bin/droid.sh")
folder_to_analyze = os.path.join(base_path, f"dcaonnextcloud-500gb/DigitalMaterialCopies/Semiramis/302_V-Zug_Documentation/{dataset_to_analyze}")
output_folder = os.path.join(base_path, f"dca-metadataraw/Semiramis/{dataset_to_analyze}_results")
output_csv_path = os.path.join(output_folder, "analysis_result_2.csv")

print(f"Analyzing folder: {folder_to_analyze}")
print(f"Output will be saved to: {output_csv_path}")

Analyzing folder: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/Semiramis/302_V-Zug_Documentation/2104_mockup
Output will be saved to: /home/renku/work/dca-metadataraw/Semiramis/2104_mockup_results/analysis_result_2.csv


## DROID-Analyse ausführen

Führt die DROID-Analyse auf dem konfigurierten Ordner aus.

In [3]:
try:
    # Output-Ordner erstellen (falls nicht vorhanden)
    os.makedirs(output_folder, exist_ok=True)

    print(f"Analyzing folder: {folder_to_analyze}")
    print(f"Output will be saved to: {output_csv_path}")

    # DROID-Skript ausführen
    result = subprocess.run(
        [droid_script_path, "-R", folder_to_analyze, "-o", output_csv_path, "-Pr", "profile.generateHash=true"],
        check=True,
        capture_output=True,
        text=True
    )

    print("DROID output:", result.stdout)
    print(f"Analysis complete. The result is saved in {output_csv_path}.")

except subprocess.CalledProcessError as e:
    print(f"An error occurred while running DROID: {e}")
    print("DROID error output:", e.stderr)

except PermissionError as e:
    print(f"Permission error: {e}")
    print("Bitte prüfe, ob du Schreibrechte für den Zielordner hast.")

except FileNotFoundError as e:
    print(f"Datei oder Verzeichnis nicht gefunden: {e}")
    print("Bitte prüfe, ob alle Pfade korrekt sind und existieren.")

Analyzing folder: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/Semiramis/302_V-Zug_Documentation/2104_mockup
Output will be saved to: /home/renku/work/dca-metadataraw/Semiramis/2104_mockup_results/analysis_result_2.csv
DROID output: 2025-11-12T12:59:47,216  INFO [main] DroidCommandLine:225 - Starting DROID.
2025-11-12T12:59:48,406  INFO [main] ProfileManagerImpl:129 - Creating profile: 1762952388406
2025-11-12T12:59:48,425  INFO [main] ProfileInstance:365 - Attempting state change [INITIALISING] to [VIRGIN]
2025-11-12T12:59:49,848 ERROR [main] WriterResultHandlerDao:172 - /home/renku/work/dca-metadataraw/Semiramis/2104_mockup_results/analysis_result_2.csv (Read-only file system)
2025-11-12T12:59:51,854  INFO [main] ProfileDiskAction:87 - Saving profile [/home/renku/.droid6/profiles/1762952388406/db] to [/home/renku/.droid6/profile_templates/schema 6.7.0/profile.template]
2025-11-12T12:59:56,380  INFO [main] ProfileDiskAction:87 - Saving profile [/home/renku/.droid6/profi

## Analyse der DROID-Ergebnisse

Die folgenden Zellen führen verschiedene Analysen auf den DROID-Ergebnissen durch.

In [20]:
%pip install pandas matplotlib pywaffle seaborn 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 6.8 MB/s eta 0:00:0000:01m00:01

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Pfad zu src/ hinzufügen, um lokale Module zu importieren
src_path = os.path.join(base_path, "27_DCA_Ingest/src")
if src_path not in sys.path:
    sys.path.append(src_path)

# Versuche, pywaffle zu installieren/importieren (falls benötigt)
try:
    from pywaffle import Waffle
    print("pywaffle ist verfügbar")
except ImportError:
    print("Hinweis: pywaffle ist nicht installiert. Waffle-Charts werden übersprungen.")
    print("Installieren Sie pywaffle mit: pip install pywaffle")

try:
    import seaborn as sns
    print("seaborn ist verfügbar")
except ImportError:
    print("Hinweis: seaborn ist nicht installiert. Heatmaps werden möglicherweise übersprungen.")
    print("Installieren Sie seaborn mit: pip install seaborn")

pywaffle ist verfügbar
seaborn ist verfügbar


In [22]:
# Pfade für die Analyse konfigurieren
# CSV_PATH: Pfad zur DROID-Ausgabe-CSV
if "output_csv_path" in locals():
    CSV_PATH = output_csv_path
else:
    # Fallback, falls output_csv_path nicht definiert wurde
    CSV_PATH = os.path.join(base_path, f"dca-metadataraw/Semiramis/{dataset_to_analyze}_results/analysis_result.csv")

# OUTPUT_DIR: Verzeichnis für Analyse-Ausgaben
OUTPUT_DIR = os.path.join(output_folder, "res")
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"CSV-Datei für Analyse: {CSV_PATH}")
print(f"Ausgabeverzeichnis für Plots: {OUTPUT_DIR}")

OSError: [Errno 30] Read-only file system: '/home/renku/work/dca-metadataraw/Semiramis/2104_mockup_results/res'

In [ ]:
# Analyse: Anzahl der Dateien pro Format
try:
    from analyse_formats import analyse_formats
    analyse_formats(CSV_PATH, OUTPUT_DIR, show_plot=False)
except ImportError:
    print("Hinweis: analyse_formats Modul nicht gefunden. Überspringe diese Analyse.")
except FileNotFoundError as e:
    print(f"Datei nicht gefunden: {e}")
except Exception as e:
    print(f"Fehler bei analyse_formats: {e}")

In [ ]:
# Analyse: Zeitliche Verteilung eines spezifischen Formats
# Beispiel: Analysiere das häufigste Format über die Zeit
try:
    from analyse_format_over_time import analyse_format_over_time
    
    # Ermittle das häufigste Format
    if os.path.exists(CSV_PATH):
        df = pd.read_csv(CSV_PATH)
        most_common_format = df["FORMAT_NAME"].value_counts().index[0]
        print(f"Analysiere zeitliche Verteilung von: {most_common_format}")
        analyse_format_over_time(CSV_PATH, most_common_format, OUTPUT_DIR, show_plot=False)
    else:
        print(f"CSV-Datei nicht gefunden: {CSV_PATH}")
except ImportError:
    print("Hinweis: analyse_format_over_time Modul nicht gefunden. Überspringe diese Analyse.")
except Exception as e:
    print(f"Fehler bei analyse_format_over_time: {e}")

In [ ]:
# Analyse: Waffle-Chart für monatliche Dateihäufigkeit
try:
    from generate_waffle import generate_waffle_chart
    
    # Ermittle den Zeitraum aus den Daten
    if os.path.exists(CSV_PATH):
        df = pd.read_csv(CSV_PATH)
        df["LAST_MODIFIED"] = pd.to_datetime(df["LAST_MODIFIED"], errors="coerce")
        df_clean = df.dropna(subset=["LAST_MODIFIED"])
        
        if len(df_clean) > 0:
            start_year = df_clean["LAST_MODIFIED"].dt.year.min()
            end_year = df_clean["LAST_MODIFIED"].dt.year.max()
            print(f"Waffle-Chart für Zeitraum: {start_year}-{end_year}")
            generate_waffle_chart(CSV_PATH, OUTPUT_DIR, start_year, end_year, top_n=10, show_plot=False)
        else:
            print("Keine gültigen Datumsangaben in den Daten gefunden.")
    else:
        print(f"CSV-Datei nicht gefunden: {CSV_PATH}")
except ImportError as e:
    print(f"Hinweis: Modul nicht gefunden ({e}). Überspringe Waffle-Chart.")
    print("Installieren Sie pywaffle falls benötigt: pip install pywaffle")
except Exception as e:
    print(f"Fehler bei generate_waffle_chart: {e}")

In [ ]:
# Analyse: Heatmap der häufigsten Formate nach Monat
try:
    from analyse_format_heatmap import generate_format_heatmap
    
    # Ermittle den Zeitraum aus den Daten
    if os.path.exists(CSV_PATH):
        df = pd.read_csv(CSV_PATH)
        df["LAST_MODIFIED"] = pd.to_datetime(df["LAST_MODIFIED"], errors="coerce")
        df_clean = df.dropna(subset=["LAST_MODIFIED"])
        
        if len(df_clean) > 0:
            start_year = df_clean["LAST_MODIFIED"].dt.year.min()
            end_year = df_clean["LAST_MODIFIED"].dt.year.max()
            print(f"Heatmap für Zeitraum: {start_year}-{end_year}")
            generate_format_heatmap(CSV_PATH, OUTPUT_DIR, start_year, end_year, top_n=10, show_plot=False)
        else:
            print("Keine gültigen Datumsangaben in den Daten gefunden.")
    else:
        print(f"CSV-Datei nicht gefunden: {CSV_PATH}")
except ImportError as e:
    print(f"Hinweis: Modul nicht gefunden ({e}). Überspringe Heatmap.")
    print("Installieren Sie seaborn falls benötigt: pip install seaborn")
except Exception as e:
    print(f"Fehler bei generate_format_heatmap: {e}")